# Retail Price Optimization

**Purpose**: Setting a price on a product is an imperative component of business and can either result in success or failure depending on how the process is handled. If executed correctly, the price of a product will not only be decided with the intent of generating as much profit as possible, but with the consideration of customer attraction as well. The retail price of a product that is more skewed toward maximizing returns will drive increased set prices which will usually decrease customer attraction, and therefore lead to low customer attractiveness to a product. The opposite is also destructive as setting too low of a price, even if customer attraction is high, results in not having a return on investment and perhaps revenue for the product running in the negative.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from scipy.stats import pearsonr
pio.renderers.default = "svg"
pio.templates.default = 'seaborn'

In [ ]:
data = pd.read_csv('retail_price.csv')

In [ ]:
column_dict = {'product_id':'A unique identifier for each product in the dataset.',
               'product_category_name': 'The name of the product category to which the product belongs.',
               'month_year': 'The month and year of the retail transaction or data recording.',
               'qty': 'The quantity of the product sold or purchased in a given transaction.',
               'total_price': 'The total price of the product, including any applicable taxes or discounts.',
               'freight_price': 'The cost of shipping or freight associated with the product.',
               'unit_price': 'The price of a single unit of the product.',
               'product_name_length': 'The length of the product name in terms of the number of characters.',
               'product_description_length': 'The length of the product description in terms of the number of characters.',
               'product_photos_qty': 'The number of photos available for the product in the dataset.',
               'product_weight_g': 'The weight of the product in grams.',
               'product_score': 'A score or rating associated with the product’s quality, popularity, or other relevant factors.',
               'customers': 'The number of customers who purchased the product in a given transaction.',
               'weekday': 'The day of the week on which the transaction occurred.',
               'weekend': 'A binary flag indicating whether the transaction occurred on a weekend (1) or not (0).',
               'holiday': 'A binary flag indicating whether the transaction occurred on a holiday (1) or not (0).',
               'month': 'The month in which the transaction occurred.',
               'year': 'The year in which the transaction occurred.',
               's': 'The effect of seasonality.',
               'comp_1, comp_2, comp_3': 'Competitor information or variables related to competitors’ prices, promotions, or other relevant factors.',
               'ps1, ps2, ps3': 'Product score or rating associated with competitors’ products.',
               'fp1, fp2, fp3': 'Freight or shipping cost associated with competitors’ products.'
              }
print('Dictionary')
print('----------')
for key, value in column_dict.items():
    print(f'{key}: {value}')
    print('')

In [ ]:
data.head()

First, we will check for null values present in the dataset.

In [ ]:
null_data = data.isnull().sum()
null_data

Null values are not present in the dataset.

We can quickly check descriptive stats for the dataset:

In [ ]:
data.describe()

Now, we will be creating a histogram plot to visualize the distribution of total price of the products:

In [ ]:
fig = px.histogram(data, x='total_price', nbins=20, title='Total Price Distribution')
fig.show()

Now, we will visualize the distribution of unit prices using a box plot:

In [ ]:
fig = px.box(data, y='unit_price', title='Unit Price Distribution')
fig.show()

We will now visualize the relationship between quantity and total price:

In [ ]:
fig = px.scatter(data, x='qty', y='total_price', title='Quantity vs Total Price', trendline = 'ols')
fig.show()

From the Quantity vs Total Price graph we observe a linear relationship between quantity and total prices.

Let's statistically verify this relationship by determining the correlation among the two variables:

In [ ]:
corr, _ = pearsonr(data['qty'], data['total_price'])
print(f'Pearson Correlation: {corr:.3f}')

A pearson correlation value of **0.750** indicates a relatively strong positive correlation between quantity and total price

Next, we will take a look at the average total prices by product categories:

In [ ]:
fig = px.bar(data, x='product_category_name', y='total_price', title='Average Total Price by Product Category')
fig.show()

We will now check out the distribution of total prices by weekday using boxplots:

In [ ]:
fig = px.box(data, x='weekday', y='total_price', title='Box Plot of Total Price by Weekday')
fig.show()

We will now check for the distribution of total prices by holiday:

In [ ]:
fig = px.box(data, x='holiday', y='total_price', title='Box Plot of Total Price by Holiday')
fig.show()

Let's take a look at the correlation between the numerical features with each other:

In [ ]:
correlation_matrix = data.corr()
fig = go.Figure(go.Heatmap(x=correlation_matrix.columns, y=correlation_matrix.columns,
                           z=correlation_matrix.values))
fig.update_layout(title='Correlation Heatmap of Numerical Features')
fig.show()

One thing to keep in mind when optimizing retail price is competitor comparisons. Other businesses have set prices as well and products must be competively priced so that consumers are more inclined to purchase a particular business's product over the other. This can be done by lowering the product price (e.g. discounts) to have the competitive edge in the overall market.

In [ ]:
data['comp_price_diff'] = data['unit_price'] - data['comp_1']

avg_price_diff_by_category = data.groupby('product_category_name')['comp_price_diff'].mean().reset_index()

fig = px.bar(avg_price_diff_by_category, x='product_category_name', y='comp_price_diff',
             title='Average Competitor Price Difference by Product Category')
fig.update_layout(xaxis_title='Product Category', yaxis_title='Average Competitor Price Difference')
fig.show()

Retail Price Optimization Model:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
X = data[['qty', 'unit_price', 'comp_1', 'product_score', 'comp_price_diff']]
y = data['total_price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Now it is time to train the linear regression model:

In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

With the model now trained, we can make predictions for retail prices compared to the actual retail prices:

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=y_pred, mode='markers', marker=dict(color='blue'),
                         name='Predicted vs Actual Retail Price'))
fig.add_trace(go.Scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)], mode='lines',
                         marker=dict(color='red'), name='Ideal Prediction'))
fig.update_layout(title='Predicted vs Actual Retail Price', xaxis_title='Actual Retail Price',
                  yaxis_title='Predicted Retail Price')
fig.show()

This machine learning model allows us to see predicted and actual retail prices that can be utilized to optimally price products. For example, a clear example would be the product that's actual price is near 12k, our model predicts the retail price should be closer to 8k suggesting that the price should be lowered to hopefully maximize profits while having satisfied customers. Using this model, prices can be adjusted accordingly in such a way that there is balance between profit and customer attraction.